<a href="https://colab.research.google.com/github/MengOonLee/Deep_learning/blob/master/TFDS/TFRecord.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import tensorflow as tf
tf.keras.utils.set_random_seed(seed=42)
tf.get_logger().setLevel("ERROR")

x = np.arange(100, dtype=np.float32)
ds = tf.data.Dataset.from_tensor_slices(tensors=x)
ds = ds.window(size=11, shift=1)
ds = ds.flat_map(lambda w: w.batch(
    batch_size=11, drop_remainder=True))
ds = ds.map(lambda w: (w[:-1], w[-1]))

next(iter(ds))

(<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 2., 3., 4., 5., 6., 7., 8., 9.], dtype=float32)>,
 <tf.Tensor: shape=(), dtype=float32, numpy=10.0>)

In [8]:
import tensorflow as tf
tf.keras.utils.set_random_seed(seed=42)
tf.get_logger().setLevel("ERROR")

@tf.py_function(Tout=tf.string)
def serialize(x, y):
        x = tf.io.serialize_tensor(x).numpy()
        x_feature = tf.train.Feature(
            bytes_list=tf.train.BytesList(value=[x]))

        y = tf.io.serialize_tensor(y).numpy()
        y_feature = tf.train.Feature(
            bytes_list=tf.train.BytesList(value=[y]))

        feature = {
            "x": x_feature,
            "y": y_feature
        }

        proto = tf.train.Example(features=tf.train.Features(feature=feature))
        return proto.SerializeToString()

with tf.io.TFRecordWriter("test.tfrecord") as writer:
    for x, y in ds:
        protos = serialize(x, y)
        writer.write(protos.numpy())

In [15]:
%%bash
du -sh 'test.tfrecord'
more 'test.tfrecord'

12K	test.tfrecord
::::::::::::::
test.tfrecord
::::::::::::::
T       _QE�
R

y


 "   A
;
x6
4
2
"(      �?   @  @@  �@  �@  �@  �@   A  A�_ԉT       _QE�
R
;
x6
4
2
"(  �?   @  @@  �@  �@  �@  �@   A  A   A

y


 "  0A�K9WT       _QE�
R
;
x6
4
2
"(   @  @@  �@  �@  �@  �@   A  A   A  0A

y


 "  @Af�ÙT       _QE�
R
;
x6
4
2
"(  @@  �@  �@  �@  �@   A  A   A  0A  @A

y


 "  PA�G_�T       _QE�
R
;
x6
4
2
"(  �@  �@  �@  �@   A  A   A  0A  @A  PA

y


 "  `A�@��T       _QE�
R
;
x6
4
2
"(  �@  �@  �@   A  A   A  0A  @A  PA  `A

y


 "  pA�b�T       _QE�
R

y


 "  �A
;
x6
4
2
"(  �@  �@   A  A   A  0A  @A  PA  `A  pA�*��T       _QE�
R
;
x6
4
2
"(  �@   A  A   A  0A  @A  PA  `A  pA  �A

y


 "  �A��v1T       _QE�
R
;
x6
4
2
"(   A  A   A  0A  @A  PA  `A  pA  �A  �A

y


 "  �A�G5T       _QE�
R

y


 "  �A
;
x

In [26]:
features = {
    "x": tf.io.FixedLenFeature(shape=(10,), dtype=tf.float32),
    "y": tf.io.FixedLenFeature(shape=(), dtype=tf.float32)
}
def _parse_function(proto):
    return tf.io.parse_single_example(serialized=proto, features=features)

protos = tf.data.TFRecordDataset(filenames=["test.tfrecord"])
parsed = protos.map(_parse_function)

for record in parsed:
    for x, y in record:
        print(x)

In [27]:
parsed

<_MapDataset element_spec={'x': TensorSpec(shape=(10,), dtype=tf.float32, name=None), 'y': TensorSpec(shape=(), dtype=tf.float32, name=None)}>

In [13]:
# features = {
#     "x": tf.io.FixedLenFeature(shape=(10,), dtype=tf.float32),
#     "y": tf.io.FixedLenFeature(shape=(), dtype=tf.float32)
# }
# def _parse_function(serialized):
#     return tf.io.parse_single_example(serialized, features=features)

ds_read = tf.data.TFRecordDataset("test.tfrecord")
parsed_ds = ds_read.map(_parse_function)

for r in parsed_ds.take(10):
    print(r)

InvalidArgumentError: {{function_node __wrapped__IteratorGetNext_output_types_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} Key: x.  Data types don't match. Data type: string but expected type: float
	 [[{{node ParseSingleExample/ParseExample/ParseExampleV2}}]] [Op:IteratorGetNext] name: 